# Libs, data importing

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Plots
# ==============================================================================
import matplotlib.pyplot as plt
import plotly.express as px
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
%matplotlib inline

# Modelling and Forecasting
# ==============================================================================
from sklearn.ensemble import RandomForestRegressor


from datetime import date
from joblib import dump, load
from rich.progress import track

# Configuration
# ==============================================================================
import warnings
import holidays
warnings.filterwarnings('ignore')



In [15]:
data_train = pd.read_csv('/home/jerzy/Documents/IndustrialML/data/train_preprocessed.csv')
data_train.drop('Unnamed: 0', axis=1, inplace=True)


In [16]:
data_test = pd.read_csv('/home/jerzy/Documents/IndustrialML/data/test.csv')

In [17]:
data_val = pd.read_csv('/home/jerzy/Documents/IndustrialML/data/valid.csv')
# drop row 0101000020E610000000000000000000000000000000000000
data_val = data_val[data_val['point'] != '0101000020E610000000000000000000000000000000000000']

In [18]:
data_unique_points = data_train['point'].unique()
test_unique_points = data_test['point'].unique()
valid_unique_points = data_val['point'].unique()

# adding datetime holidays, weekends, weekdays

Add datetime column to test and valid datasets

In [19]:
data_test['datetime'] = pd.to_datetime(data_test['hour'], unit='s')
data_val['datetime'] = pd.to_datetime(data_val['hour'], unit='s')
data_train['datetime'] = pd.to_datetime(data_train['time'])

In [20]:
data_train

,point,lon,lat,time,timestamp,num_posts,datetime
0,0101000020E610000002A5EC7AB31D3E4097654065F8EA...,30.116020,59.835705,2019-11-22 11:00:00,1574420400,1,2019-11-22 11:00:00
1,0101000020E610000002A5EC7AB31D3E4097654065F8EA...,30.116020,59.835705,2019-11-22 12:00:00,1574424000,1,2019-11-22 12:00:00
2,0101000020E610000002A5EC7AB31D3E4097654065F8EA...,30.116020,59.835705,2019-11-23 13:00:00,1574514000,1,2019-11-23 13:00:00
3,0101000020E610000002A5EC7AB31D3E4097654065F8EA...,30.116020,59.835705,2019-11-25 16:00:00,1574697600,1,2019-11-25 16:00:00
4,0101000020E610000002A5EC7AB31D3E4097654065F8EA...,30.116020,59.835705,2019-11-27 10:00:00,1574848800,1,2019-11-27 10:00:00
...,...,...,...,...,...,...,...
3625651,0101000020E6100000FF9D4C0EC3853E4094782B2D0DF3...,30.522508,59.898840,2019-12-06 04:00:00,3151209600,2,2019-12-06 04:00:00
3625652,0101000020E6100000FF9D4C0EC3853E4094782B2D0DF3...,30.522508,59.898840,2019-12-23 08:00:00,1577088000,1,2019-12-23 08:00:00
3625653,0101000020E6100000FF9D4C0EC3853E40ABD94A1972EF...,30.522508,59.870670,2019-06-10 09:00:00,1560157200,1,2019-06-10 09:00:00
3625654,0101000020E6100000FF9D4C0EC3853E40ABD94A1972EF...,30.522508,59.870670,2019-10-06 14:00:00,1570370400,1,2019-10-06 14:00:00


Add weekday to test and valid

In [21]:
data_train["dayhour"] = data_train["datetime"].dt.hour
data_train["weekday"] = data_train["datetime"].dt.weekday

data_test["dayhour"] = data_test["datetime"].dt.hour
data_test["weekday"] = data_test["datetime"].dt.weekday

data_val["dayhour"] = data_val["datetime"].dt.hour
data_val["weekday"] = data_val["datetime"].dt.weekday

Add holidays

In [22]:
holidays_russia = holidays.country_holidays('RU', years = [2019, 2020])

data_train['is_holiday'] = data_train['datetime'].apply(lambda x: 1 if x in holidays_russia else 0)
data_test['is_holiday'] = data_test['datetime'].apply(lambda x: 1 if x in holidays_russia else 0)
data_val['is_holiday'] = data_val['datetime'].apply(lambda x: 1 if x in holidays_russia else 0)

In [23]:
data_train = data_train.drop(['time'], axis=1)

# Random Forest solution

In [24]:
def custom_metric(y_true, y_pred):
    return abs(y_true - y_pred) / y_true

def flatten(l):
    return [item for sublist in l for item in sublist]

def rfr_model_fit_predict(data_train: pd.DataFrame, data_test: pd.DataFrame, model):

    data_test_rf_error = []
    test_unique_points = data_test['point'].unique()

    for point in track(test_unique_points, description='Fitting and predicting'):
        data_train_point = data_train[data_train['point'] == point]
        data_test_point = data_test[data_test['point'] == point]

        X_train = data_train_point[['timestamp', 'dayhour', 'weekday', 'is_holiday']]
        y_train = data_train_point['num_posts']
        X_test = data_test_point[['hour', 'dayhour', 'weekday', 'is_holiday']]
        y_test = data_test_point['sum']

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        data_test_rf_error.append(custom_metric(y_test, y_pred))
    
    return data_test_rf_error

### test data

time = 203 s

error = 0.8662

In [25]:
%%time

data_test_rfr_error = rfr_model_fit_predict(data_train, data_test, RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42))
data_test_rfr_error = flatten(data_test_rfr_error)
data_test['rfr_error'] = data_test_rfr_error
data_test['rfr_error'].mean()

Output()

CPU times: user 3min 14s, sys: 186 ms, total: 3min 14s
Wall time: 3min 11s


0.8662030853982366

### validation data

time = 195 s

error = 0.86883

In [26]:
%%time
data_val_rfr_error = rfr_model_fit_predict(data_train, data_val, RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42))
data_val_rfr_error = flatten(data_val_rfr_error)
data_val['rfr_error'] = data_val_rfr_error
data_val['rfr_error'].mean()

Output()

CPU times: user 3min 15s, sys: 252 ms, total: 3min 16s
Wall time: 3min 13s


0.8688384729816974